### Table 1: BaLu_GRAPE (rconv=GCN)

| p_miss | Instagram_PEHE | Instagram_MAE | Youtube_PEHE | Youtube_MAE | BlogCatalog_PEHE | BlogCatalog_MAE | Flickr_PEHE | Flickr_MAE |
|--------|----------------|----------------|--------------|--------------|-------------------|------------------|-------------|-------------|
| 0.0    | 0.80 ± 0.03     | 0.05 ± 0.04     | 0.79 ± 0.02   | 0.03 ± 0.03   | 7.30 ± 3.72        | 0.93 ± 0.47       | 7.25 ± 1.73  | 1.03 ± 0.46  |
| 0.1    | 1.35 ± 0.25     | 0.33 ± 0.18     | 1.23 ± 0.11   | 0.13 ± 0.05   | 8.59 ± 5.56        | 0.80 ± 0.50       | 7.93 ± 2.75  | 0.72 ± 0.37  |
| 0.3    | 2.27 ± 0.71     | 1.14 ± 0.78     | 1.93 ± 0.21   | 0.23 ± 0.09   | 8.60 ± 5.15        | 0.85 ± 0.68       | 8.53 ± 1.73  | 0.71 ± 0.60  |

---

### Table 2: BaLu_IGMC (conv=GCN, rconv=GCN)

| p_miss | Instagram_PEHE | Instagram_MAE | Youtube_PEHE | Youtube_MAE | BlogCatalog_PEHE | BlogCatalog_MAE | Flickr_PEHE | Flickr_MAE |
|--------|----------------|----------------|--------------|--------------|-------------------|------------------|-------------|-------------|
| 0.0    | 0.80 ± 0.02     | 0.06 ± 0.03     | 0.79 ± 0.02   | 0.03 ± 0.02   | 7.45 ± 3.96        | 0.90 ± 0.61       | 7.44 ± 1.76  | 1.09 ± 0.38  |
| 0.1    | 1.31 ± 0.19     | 0.32 ± 0.20     | 1.22 ± 0.12   | 0.13 ± 0.07   | 8.49 ± 4.63        | 0.54 ± 0.34       | 7.99 ± 1.64  | 0.93 ± 0.80  |
| 0.3    | 2.17 ± 0.67     | 1.10 ± 0.81     | 1.89 ± 0.18   | 0.25 ± 0.18   | 10.34 ± 6.24       | 1.06 ± 0.58       | 7.97 ± 0.94  | 0.64 ± 0.75  |


## 

In [1]:
import pandas as pd
import os
import json
import numpy as np


def collect_data_p(p: str, datasets=[], result_dir='results', train_test_flag="test"):
    """
    Collects data from multiple datasets and methods for a specific missingness percentage,
    and formats it into a DataFrame.
    
    Args:
        p (str): The missingness percentage (e.g., "0.0").
        datasets (list): List of dataset names to analyze.
        output_dir (str): The base directory where results are saved.
        
    Returns:
        pd.DataFrame: A DataFrame containing formatted results for each method and imputation.
    """
    if not datasets:
        datasets = [d for d in os.listdir(result_dir) if os.path.isdir(os.path.join(result_dir, d))]
    
    for dataset in datasets[1:]:
        dataset_dir = os.path.join(result_dir, dataset)
        method_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        break
    # print(method_dirs)
    method_dirs = [e for e in method_dirs if 'Balu' in e] # or 'gnn' in e or '9' in e or '10' in e]
    print("methods:", method_dirs)

    def fileter_method(must_item: list, method):
        method_name = method
        for item in must_item:
            if item not in method:
                return False, None
            if "L=" in item or 'Balu_plus' in item:
                item = item 
            else:
                item = "_"+item
            method_name = method_name.replace(item, "")
        return True, method_name
    # 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001'

    data_rows = []
    for method in method_dirs:
        flag, method_name = fileter_method(['Balu_plus_imp=', 'L=64-64_K=64', "reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001"], method)
        if not flag:
            continue
        if "-64" in method_name:
            continue
        if "BaLu_GRAPE_gconv" not in method_name and "BaLu_IGMC_gconv" not in method_name:
            continue

        row = {
            'method': method_name,
            # 'imputation': imputation
        }
        
        # Collect results for each dataset
        for dataset in datasets:
            dataset_dir = os.path.join(result_dir, dataset)
            if not os.path.exists(dataset_dir):
                print(f"{dataset_dir} not exits!")
                # Dataset doesn't exist, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
                
            method_dir = os.path.join(dataset_dir, method)
            if not os.path.exists(method_dir):
                # Method doesn't exist for this dataset, fill with N/A
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
            
            imputation_dir = method_dir
            
            results_files = [f for f in os.listdir(imputation_dir) 
                        if f.startswith(f"p={p}_") and f.endswith(f"_{train_test_flag}_results.json")]
            if "complete" not in train_test_flag:
                results_files = [f for f in results_files if "complete" not in f]
            # results_files=results_files[:len(results_files)*1.5//2]

            # print(results_files)
            effect_pehe_values = []
            effect_mae_values = []
            effect_sum_values = []
            
            for file in results_files:
                file_path = os.path.join(method_dir, file) # imputation, file)
                try:
                    with open(file_path, 'r') as f:
                        results = json.load(f)
                        if 'effect_pehe' in results:
                            effect_pehe_values.append(results['effect_pehe'])
                        if 'effect_mae' in results:
                            effect_mae_values.append(results['effect_mae'])

                        effect_sum_values.append(results['effect_pehe'] + results['effect_mae'])
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f"Warning: Error reading file {file_path}: {e}")
            
            if effect_pehe_values:
                mean_pehe = np.mean(effect_pehe_values)
                std_pehe = np.std(effect_pehe_values)
                row[f"{dataset}_PEHE"] = f"{mean_pehe:.2f} ± {std_pehe:.2f}"
            else:
                row[f"{dataset}_PEHE"] = "N/A"
            
            if effect_mae_values:
                mean_mae = np.mean(effect_mae_values)
                std_mae = np.std(effect_mae_values)
                row[f"{dataset}_MAE"] = f"{mean_mae:.2f} ± {std_mae:.2f}"
            else:
                row[f"{dataset}_MAE"] = "N/A"

            # if effect_sum_values:
            #     mean_mae = np.mean(effect_sum_values)
            #     std_mae = np.std(effect_sum_values)
            #     row[f"{dataset}_SUM"] = f"{method} {mean_mae:.2f} ± {std_mae:.2f}"
            # else:
            #     row[f"{dataset}_SUM"] = 'N/A'
        data_rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data_rows)
    # print(df)
    
    # Sort the DataFrame by method and imputation
    if not df.empty:
        df = df.sort_values(by=['method']) #, 'imputation'])
    return df

def display_results(result_dir, p, datasets=["Syn"], train_test_flag='test', metrics=['SUM']):
    

    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)  # Show all rows
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)  # Use the full width of the notebook cell
    pd.set_option('display.max_colwidth', None)  # Show full content of each column

    # Example: Display the DataFrame created by collect_data_p
    df = collect_data_p(p, datasets=datasets, result_dir=result_dir, train_test_flag=train_test_flag) #, "AMZS", "Flickr"])
    dataset_map = {'Syn_M=None_SimRel=1_Rel=4':"Instagram",  'BlogCatalog1_M=20_SimRel=0_Rel=1':"BlogCatalog", 'BlogCatalog1_M=20_SimRel=1_Rel=4':"BlogCatalog", 'Flickr1_M=20_SimRel=0_Rel=1':"Flickr", 'Flickr1_M=20_SimRel=1_Rel=4':"Flickr", 'Youtube_M=20_SimRel=1_Rel=4':"Youtube"}
    
    def replace_col(col):
        for k, v in dataset_map.items():
            if k in col:

                return col.replace(k, v).replace("_MCAR","")
        return col

    new_column_names = [replace_col(col) for col in df.columns]
    df.columns = new_column_names
    display(df)
    # print(df.to_latex(index=False, escape=False))
    # print(df.to_markdown(index=False))

# display_results(p='0.5', train_test_flag='test')

In [2]:
datasets = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']     # , 'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4'
datasets = [e+"_MCAR" for e in datasets] 
result_dir = 'results_balu_tuning'

display_results(result_dir=result_dir, p='0.0', datasets=datasets, train_test_flag='test')
display_results(result_dir=result_dir, p='0.1', datasets=datasets, train_test_flag='test')
display_results(result_dir=result_dir, p='0.3', datasets=datasets, train_test_flag='test')
# display_results(result_dir='results_Q1', p='0.5', datasets=datasets, train_test_flag='test')


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
9,BaLu_GRAPE_gconv=GCN_rconv=GAT,0.78 ± 0.02,0.04 ± 0.02,0.79 ± 0.02,0.02 ± 0.02,11.66 ± 9.29,1.66 ± 1.28,15.30 ± 2.77,3.18 ± 2.37
8,BaLu_GRAPE_gconv=GCN_rconv=GCN,0.80 ± 0.03,0.05 ± 0.04,0.79 ± 0.02,0.03 ± 0.03,7.30 ± 3.72,0.93 ± 0.47,7.25 ± 1.73,1.03 ± 0.46
3,BaLu_GRAPE_gconv=GCN_rconv=GraphSAGE,0.79 ± 0.02,0.06 ± 0.04,0.79 ± 0.02,0.03 ± 0.02,12.46 ± 10.33,1.79 ± 1.16,15.15 ± 2.39,3.09 ± 2.25
4,BaLu_GRAPE_gconv=GCN_rconv=RGCN,0.79 ± 0.02,0.05 ± 0.03,0.79 ± 0.02,0.03 ± 0.02,11.91 ± 9.96,1.85 ± 1.15,15.09 ± 2.53,3.18 ± 2.23
1,BaLu_IGMC_gconv=GAT_rconv=GAT,0.79 ± 0.02,0.06 ± 0.06,0.79 ± 0.02,0.02 ± 0.02,11.39 ± 8.27,1.85 ± 1.12,15.05 ± 2.36,3.17 ± 2.34
5,BaLu_IGMC_gconv=GAT_rconv=RGCN,0.80 ± 0.02,0.04 ± 0.03,0.79 ± 0.02,0.03 ± 0.02,11.82 ± 9.73,1.73 ± 1.22,14.93 ± 2.27,3.04 ± 2.21
7,BaLu_IGMC_gconv=GCN_rconv=GCN,0.80 ± 0.02,0.06 ± 0.03,0.79 ± 0.02,0.03 ± 0.02,7.45 ± 3.96,0.90 ± 0.61,7.44 ± 1.76,1.09 ± 0.38
2,BaLu_IGMC_gconv=GCN_rconv=RGCN,0.80 ± 0.03,0.07 ± 0.05,0.80 ± 0.02,0.03 ± 0.02,11.24 ± 8.13,1.83 ± 1.24,15.15 ± 2.66,3.11 ± 2.40
6,BaLu_IGMC_gconv=GraphSAGE_rconv=GraphSAGE,0.79 ± 0.02,0.04 ± 0.02,0.78 ± 0.02,0.03 ± 0.01,12.02 ± 10.14,1.87 ± 1.21,15.32 ± 2.45,3.11 ± 2.34
0,BaLu_IGMC_gconv=GraphSAGE_rconv=RGCN,0.79 ± 0.02,0.04 ± 0.03,0.79 ± 0.02,0.02 ± 0.02,11.72 ± 9.17,1.75 ± 1.24,14.99 ± 2.17,3.11 ± 2.29


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
9,BaLu_GRAPE_gconv=GCN_rconv=GAT,1.84 ± 0.18,0.32 ± 0.19,1.93 ± 0.20,0.12 ± 0.04,9.95 ± 3.97,1.59 ± 1.16,15.49 ± 2.43,2.75 ± 2.17
8,BaLu_GRAPE_gconv=GCN_rconv=GCN,1.88 ± 0.23,0.31 ± 0.19,1.91 ± 0.20,0.12 ± 0.06,7.64 ± 4.26,0.92 ± 0.52,7.94 ± 2.85,0.59 ± 0.38
3,BaLu_GRAPE_gconv=GCN_rconv=GraphSAGE,1.85 ± 0.18,0.34 ± 0.22,1.91 ± 0.20,0.13 ± 0.09,9.20 ± 2.49,1.70 ± 1.31,15.26 ± 2.40,2.72 ± 2.07
4,BaLu_GRAPE_gconv=GCN_rconv=RGCN,1.89 ± 0.24,0.34 ± 0.25,1.92 ± 0.21,0.10 ± 0.06,9.20 ± 2.81,1.70 ± 1.13,15.27 ± 2.51,2.73 ± 1.98
1,BaLu_IGMC_gconv=GAT_rconv=GAT,1.87 ± 0.21,0.30 ± 0.25,1.92 ± 0.20,0.11 ± 0.07,9.92 ± 4.50,1.77 ± 1.21,15.38 ± 2.49,2.75 ± 1.91
5,BaLu_IGMC_gconv=GAT_rconv=RGCN,1.83 ± 0.19,0.32 ± 0.22,1.92 ± 0.20,0.11 ± 0.08,9.71 ± 3.28,1.77 ± 1.16,15.18 ± 2.50,2.75 ± 1.78
7,BaLu_IGMC_gconv=GCN_rconv=GCN,1.85 ± 0.20,0.29 ± 0.19,1.91 ± 0.21,0.10 ± 0.08,8.16 ± 4.28,0.62 ± 0.33,8.00 ± 1.79,0.87 ± 0.82
2,BaLu_IGMC_gconv=GCN_rconv=RGCN,1.85 ± 0.19,0.30 ± 0.26,1.93 ± 0.21,0.10 ± 0.06,9.60 ± 3.62,1.71 ± 1.17,15.25 ± 2.39,2.59 ± 2.11
6,BaLu_IGMC_gconv=GraphSAGE_rconv=GraphSAGE,1.86 ± 0.21,0.30 ± 0.22,1.92 ± 0.20,0.11 ± 0.07,10.51 ± 5.47,1.77 ± 1.25,15.09 ± 2.43,2.81 ± 1.93
0,BaLu_IGMC_gconv=GraphSAGE_rconv=RGCN,1.88 ± 0.19,0.35 ± 0.23,1.92 ± 0.21,0.12 ± 0.05,10.63 ± 5.65,1.71 ± 1.21,15.44 ± 2.83,2.66 ± 2.07


methods: ['Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GraphSAGE_rconv=RGCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GAT_rconv=GAT_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.01_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.1_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64-64_K=64-64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=IGMCL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.0001_eta=0.0001', 'Balu_plus_imp=BaLu_GRAPEL=64-64_K=64_gconv=GCN_rconv=GCN_reldrop=0.0_beta=0.0001_gamma=0.1_eta=0.0001', 'Balu_plus_imp=BaLu_IGM

,method,Instagram_PEHE,Instagram_MAE,Youtube_PEHE,Youtube_MAE,BlogCatalog_PEHE,BlogCatalog_MAE,Flickr_PEHE,Flickr_MAE
9,BaLu_GRAPE_gconv=GCN_rconv=GAT,3.12 ± 0.60,1.06 ± 0.79,3.15 ± 0.37,0.24 ± 0.19,10.52 ± 3.69,1.90 ± 1.23,15.54 ± 2.18,2.56 ± 1.75
8,BaLu_GRAPE_gconv=GCN_rconv=GCN,3.14 ± 0.60,1.09 ± 0.78,3.13 ± 0.36,0.23 ± 0.13,7.65 ± 3.93,1.23 ± 0.50,7.24 ± 0.89,0.58 ± 0.45
3,BaLu_GRAPE_gconv=GCN_rconv=GraphSAGE,3.10 ± 0.57,1.04 ± 0.74,3.17 ± 0.37,0.28 ± 0.15,9.50 ± 2.70,2.24 ± 1.10,15.32 ± 2.39,2.59 ± 1.47
4,BaLu_GRAPE_gconv=GCN_rconv=RGCN,3.08 ± 0.53,0.99 ± 0.72,3.18 ± 0.38,0.21 ± 0.14,10.41 ± 3.45,2.33 ± 1.21,15.52 ± 2.46,2.68 ± 1.46
1,BaLu_IGMC_gconv=GAT_rconv=GAT,3.14 ± 0.56,1.08 ± 0.76,3.15 ± 0.37,0.25 ± 0.15,9.99 ± 3.19,2.14 ± 1.37,15.38 ± 2.35,2.46 ± 0.95
5,BaLu_IGMC_gconv=GAT_rconv=RGCN,3.06 ± 0.56,1.02 ± 0.76,3.14 ± 0.42,0.21 ± 0.16,10.51 ± 3.22,2.31 ± 1.30,15.47 ± 2.44,2.64 ± 1.42
7,BaLu_IGMC_gconv=GCN_rconv=GCN,3.10 ± 0.60,1.08 ± 0.81,3.12 ± 0.34,0.22 ± 0.17,9.44 ± 5.64,1.65 ± 0.79,7.46 ± 1.14,0.73 ± 0.55
2,BaLu_IGMC_gconv=GCN_rconv=RGCN,3.08 ± 0.55,1.01 ± 0.69,3.14 ± 0.39,0.29 ± 0.21,9.92 ± 3.29,2.18 ± 1.11,15.52 ± 2.21,2.85 ± 1.19
6,BaLu_IGMC_gconv=GraphSAGE_rconv=GraphSAGE,3.11 ± 0.61,1.06 ± 0.80,3.15 ± 0.39,0.28 ± 0.16,10.02 ± 3.11,2.17 ± 1.20,15.64 ± 2.45,2.71 ± 1.45
0,BaLu_IGMC_gconv=GraphSAGE_rconv=RGCN,3.09 ± 0.58,1.03 ± 0.74,3.15 ± 0.36,0.25 ± 0.20,10.03 ± 3.09,2.14 ± 1.33,15.50 ± 2.63,2.51 ± 1.51
